In [2]:
from soma import aims
import pandas as pd
import numpy as np
from pathlib import Path

from typing import Dict

# Paths for the analysis

<span style="color:red"> Analysis on V1 subject in ataxia, and on PN (it's a preprocessing done on the data -> cf. Clara)</span>

In [3]:
class PathSubjectAtaxia : 
    def __init__(self, subject_id): 
        # Paths template
        # DB location
        ATAXIA_ZSUN = Path("/neurospin/dico/zsun/ataxie/etudes_AlexandraDurr/database_brainvisa/cermoi")
        CATI_CERMOI_DB = Path("/neurospin/cati/cati_members/studies_cati/cermoi/database_brainvisa/00")

        # Tree in the DB to access the graphs 
        # Ataxia zsun
        TREE_GRAPH = Path("t1mri/V1/default_analysis/folds/3.1")
        # Cati processed
        TREE_CERES = Path("cereb_bs4/V1")
        NOMENCLATURE_CEREBELLUM_SEG = "_cerebellum_brainstem_split_mask.nii.gz"

        # subject specific paths
        self.id = subject_id
        self.graph = ATAXIA_ZSUN / subject_id / TREE_GRAPH / f"R{subject_id}.arg"
        self.cerebellum_mask = CATI_CERMOI_DB / subject_id / TREE_CERES / f"{subject_id}{NOMENCLATURE_CEREBELLUM_SEG}"

In [4]:
SUBJECT = "00001PJ"
sub_path = PathSubjectAtaxia(SUBJECT)
sub_path.graph, sub_path.cerebellum_mask

(PosixPath('/neurospin/dico/zsun/ataxie/etudes_AlexandraDurr/database_brainvisa/cermoi/00001PJ/t1mri/V1/default_analysis/folds/3.1/R00001PJ.arg'),
 PosixPath('/neurospin/cati/cati_members/studies_cati/cermoi/database_brainvisa/00/00001PJ/cereb_bs4/V1/00001PJ_cerebellum_brainstem_split_mask.nii.gz'))

# Retrieving cerebllum mask from CERES

We are working with the segmentation that has been done with by Clara Fischer in the ataxia project.

The segmentation has been done using Ceres for the lobes of the brain and in addition to that, the vermis have segmented using vol2brain, those 2 segmentation have been merged to have those parts : 
- Two hemispheres (separated components)
- Vermis 
- Brain stem structure (3 different components)

Only the 2 cerebellum hemisphere and the vermis are interesting for us.

### Cerebellum mask

In [5]:
aims_obj = aims.read(str(sub_path.cerebellum_mask))
vol_np = aims_obj.np
np.unique(vol_np)

array([0, 1, 2, 3, 5, 6, 7, 8], dtype=int16)

Detail of the encoding of the different parts : 
- `0` : Background
- `1` : Hemisphere
- `2` : Hemisphere
- `3` : Vermis 
- `[5-8]` : Brainstem

In [6]:
# Changing the values of the volume
def only_mask_cerebellum(vol : np.ndarray) -> None :
    cerebellum = np.isin(vol_np, [1,2,3])
    rest = np.isin(vol_np, [0,5,6,7,8])

    vol_np[cerebellum] = 1
    vol_np[rest] = 0

In [7]:
only_mask_cerebellum(vol_np)
np.unique(vol_np)

array([0, 1], dtype=int16)

In [8]:
SAVING_FOLDER = Path("/neurospin/tmp/tsanchez/cerebellum_mask/ataxia")
# aims.write(obj = aims_obj, filename= str(SAVING_FOLDER / f"{SUBJECT}_cereb_bs4_cerebellum_only.nii.gz"))

### Retreiving transform matrix to ICBM2009c w/ `deep_folding`

We need to transform to register the mask we obtained to IBCM2009c

In [9]:
def get_ICBM2009c_transform(graph_path) : 
    graph = aims.read(str(graph_path))
    transf = aims.GraphManip.getICBM2009cTemplateTransform(graph)
    return transf

In [10]:
mask_transform = get_ICBM2009c_transform(sub_path.graph)

In [11]:
from deep_folding.brainvisa.utils.resample import resample

In [12]:
RESAMPLE_VALUES = [1]
OUTPUT_VOXEL_SIZE = (1,1,1)
resampled_cereb_mask = resample(
    input_image=aims_obj, 
    transformation=mask_transform,
    output_vs=OUTPUT_VOXEL_SIZE,
    background=0,
    values= RESAMPLE_VALUES,
    verbose=True
)

DEBUG:resample.py: Time before resampling: 0.17847585678100586s
DEBUG:resample.py: Background resampling: 0.01292562484741211s
DEBUG:resample.py: Time: 1.0110745429992676s
DEBUG:resample.py: 	0.1901569366455078s to create the bucket
	0.520160436630249s to resample bucket
	0.03344130516052246s to assign values


restoring values


In [13]:
import deep_folding.brainvisa.utils.dilate_mask as dl 

DILATATION = 5 #mm
dilated_mask = dl.dilate(resampled_cereb_mask, radius=DILATATION)
aims.write(dilated_mask, filename = str(SAVING_FOLDER / f"{SUBJECT}_ICBM2009c_cerebellum_mask_{DILATATION}mm_dilatation.nii.gz"))

### Register pclean and hcp subjects to ICBM2009c

In [14]:
class PathSubject:
    def __init__(self, subject_id : str, db : str):
        assert db in ["hcp", "pclean"]

        PATH_DATA = Path("/neurospin/dico/data/bv_databases/human")
        PATH_LOCAL = Path(f"/neurospin/tmp/tsanchez/{db}")


        if db == "pclean" :
            PATH_DB = PATH_DATA / "manually_labelled" / "pclean" / "all" 
            TREE_GRAPH = Path("t1mri/t1/default_analysis/folds/3.1")
        else :
            PATH_DB = PATH_DATA / "automatically_labeled" / "hcp"/ "hcp"
            TREE_GRAPH = Path("t1mri/BL/default_analysis/folds/3.1")

        self.id = subject_id
        self.local = PATH_LOCAL
        self.graph = PATH_DB / subject_id / TREE_GRAPH / f"R{subject_id}.arg"
        self.thresh = PATH_LOCAL / "thresh" / f"threshold_cerebellum_{subject_id}.nii.gz"
        self.ICBM2009c = PATH_LOCAL / "ICBM2009c_tresh" / f"{subject_id}_ICBM2009c_thresh.nii.gz"
    
    def __repr__(self) : 
        return f"Paths({self.id})"

In [15]:
test = 518746
path_test = PathSubject(str(test), db = "hcp")
path_test.id, path_test.local, path_test.graph, path_test.thresh, path_test.ICBM2009c


('518746',
 PosixPath('/neurospin/tmp/tsanchez/hcp'),
 PosixPath('/neurospin/dico/data/bv_databases/human/automatically_labeled/hcp/hcp/518746/t1mri/BL/default_analysis/folds/3.1/R518746.arg'),
 PosixPath('/neurospin/tmp/tsanchez/hcp/thresh/threshold_cerebellum_518746.nii.gz'),
 PosixPath('/neurospin/tmp/tsanchez/hcp/ICBM2009c_tresh/518746_ICBM2009c_thresh.nii.gz'))

In [16]:
### Subjects 
dict_subjects = {
                "hcp" : [
                        "163331", 
                        "518746", 
                        "991267", 
                        ],
                "pclean" : [
                        "s12158", 
                        "s12401", 
                        "s12635", 
                        ]
                }

dict_path = {
    "hcp" : [PathSubject(sub_id, "hcp") for sub_id in dict_subjects["hcp"]],
    "pclean" : [PathSubject(sub_id, "pclean") for sub_id in dict_subjects["pclean"]],
}

dict_path

{'hcp': [Paths(163331), Paths(518746), Paths(991267)],
 'pclean': [Paths(s12158), Paths(s12401), Paths(s12635)]}

In [21]:
def transform_ICBM2009c(paths : PathSubject, save : bool = False) : 

    # TODO : Add this as arguments
    RESAMPLE_VALUES = [0, -1, 1]
    OUTPUT_VOXEL_SIZE = (1,1,1)

    # !!! Not really the native object here
    native_obj = aims.read(str(paths.thresh))

    # Apply transform
    transf = get_ICBM2009c_transform(paths.graph)
    resampled_to_ICBM2009c = resample(
        input_image=native_obj, 
        transformation=transf,
        output_vs=OUTPUT_VOXEL_SIZE,
        background=0,
        values= RESAMPLE_VALUES,
        verbose=True
    )

    if save : 
        aims.write(resampled_to_ICBM2009c, filename=str(paths.ICBM2009c))

    return resampled_to_ICBM2009c


In [22]:
test_sub = dict_path["hcp"][0]
test_sub

Paths(163331)

In [23]:
transform_ICBM2009c(paths=test_sub, save= True)

DEBUG:resample.py: Time before resampling: 0.07094812393188477s
DEBUG:resample.py: Background resampling: 0.02017974853515625s


TypeError: RawConverter_rc_ptr_Volume_S16_BucketMap_VOID.convert(): argument 1 has unexpected type 'Volume_FLOAT'